In [1]:

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.nn import functional as F
import struct
import numpy as np
import os
import random

# Constants for the models
input_dim = 28 * 28
hidden_dim_1 = 256
hidden_dim_2 = 128
categorical_dim = 10  # Number of categories in the categorical distribution (One for each digit)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the Encoder model
class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim_1, hidden_dim_2, categorical_dim):
        super(Encoder, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim_1)
        self.fc2 = nn.Linear(hidden_dim_1, hidden_dim_2)
        self.fc3 = nn.Linear(hidden_dim_2, categorical_dim)
        
    def forward(self, x):
        x = x.view(-1, input_dim)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.softmax(self.fc3(x), dim=1)
        return x

# Define the Decoder model
class Decoder(nn.Module):
    def __init__(self, categorical_dim, hidden_dim_2, hidden_dim_1, input_dim):
        super(Decoder, self).__init__()
        self.fc1 = nn.Linear(categorical_dim, hidden_dim_2)
        self.fc2 = nn.Linear(hidden_dim_2, hidden_dim_1)
        self.fc3 = nn.Linear(hidden_dim_1, input_dim)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x.view(-1, 1, 28, 28)

# Signalling game loss
def signalling_game_loss(original_image, decoded_image):
    return F.mse_loss(decoded_image, original_image)

# Growth rule
def grow_population(senders, receivers):
    # New sender inherits strategy from a random existing sender with potential small mutation (omitted for simplicity here)
    new_sender = Encoder(input_dim, hidden_dim_1, hidden_dim_2, categorical_dim).to(device)
    new_sender.load_state_dict(random.choice(senders).state_dict())
    
    # New receiver inherits strategy from a random existing receiver with potential small mutation (omitted for simplicity here)
    new_receiver = Decoder(categorical_dim, hidden_dim_2, hidden_dim_1, input_dim).to(device)
    new_receiver.load_state_dict(random.choice(receivers).state_dict())
    
    senders.append(new_sender)
    receivers.append(new_receiver)


In [2]:
# Constants for pre-training
pretrain_epochs = 10
pretrain_batch_size = 64
pretrain_learning_rate = 0.001

# Create data loader for pre-training
# Data loaders for the MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
pretrain_loader = DataLoader(train_dataset, batch_size=pretrain_batch_size, shuffle=True)

# Initialize a single Encoder and Decoder for pre-training
encoder_pretrain = Encoder(input_dim, hidden_dim_1, hidden_dim_2, categorical_dim).to(device)
decoder_pretrain = Decoder(categorical_dim, hidden_dim_2, hidden_dim_1, input_dim).to(device)

# Define optimizer and loss function for pre-training
optimizer_pretrain = optim.Adam(list(encoder_pretrain.parameters()) + list(decoder_pretrain.parameters()), lr=pretrain_learning_rate)

# Pre-training loop
for epoch in range(pretrain_epochs):
    total_loss = 0.0
    for batch_idx, (data, _) in enumerate(pretrain_loader):
        data = data.to(device)
        
        # Zero the parameter gradients
        optimizer_pretrain.zero_grad()
        
        # Forward pass: Encode and Decode
        encoded_data = encoder_pretrain(data)
        decoded_data = decoder_pretrain(encoded_data)
        
        # Calculate loss and backpropagate
        loss = signalling_game_loss(data, decoded_data)
        loss.backward()
        optimizer_pretrain.step()
        
        total_loss += loss.item()
    
    # Print epoch statistics
    average_loss = total_loss / len(pretrain_loader)
    print(f"Epoch {epoch + 1}/{pretrain_epochs}, Loss: {average_loss:.4f}")


Epoch 1/10, Loss: 0.9481
Epoch 2/10, Loss: 0.9253
Epoch 3/10, Loss: 0.9253
Epoch 4/10, Loss: 0.9253
Epoch 5/10, Loss: 0.9253
Epoch 6/10, Loss: 0.9253
Epoch 7/10, Loss: 0.9253
Epoch 8/10, Loss: 0.9253
Epoch 9/10, Loss: 0.9253
Epoch 10/10, Loss: 0.9253


In [3]:

# Dynamic population interaction and evolution
# Constants for data loading
batch_size = 64
learning_rate = 0.001

train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)  # Set batch size to 1 for manual interaction

# Constants for the dynamic interaction
initial_population = 3
num_generations_dynamic = 50
num_interactions_dynamic = 100

# Initialize a small population of agents
senders_population = [Encoder(input_dim, hidden_dim_1, hidden_dim_2, categorical_dim).to(device) for _ in range(initial_population)]
receivers_population = [Decoder(categorical_dim, hidden_dim_2, hidden_dim_1, input_dim).to(device) for _ in range(initial_population)]

# Reset the performance and population size tracking
performance_over_generations_dynamic = []
population_size_over_generations = []

# Main loop for generations with dynamic population growth
for generation in range(num_generations_dynamic):
    successful_interactions = 0
    
    # Dynamic interactions within a generation
    for interaction in range(num_interactions_dynamic):
        # Randomly select a sender and a receiver from the current population
        selected_sender = random.choice(senders_population)
        selected_receiver = random.choice(receivers_population)
        
        # Sample a random image
        sample_image, _ = next(iter(train_loader))
        sample_image = sample_image[0].to(device)
        
        # Sender encodes the image
        signal = selected_sender(sample_image.unsqueeze(0))
        
        # Receiver decodes the signal
        decoded_image = selected_receiver(signal)
        
        # Check if the communication was successful
        loss = signalling_game_loss(sample_image, decoded_image)
        if loss < 0.1:  # Using a threshold to determine success
            successful_interactions += 1
            
        # Update the selected sender and receiver based on the result
        optimizer_selected = optim.Adam(list(selected_sender.parameters()) + list(selected_receiver.parameters()), lr=learning_rate)
        optimizer_selected.zero_grad()
        loss.backward()
        optimizer_selected.step()
    
    # Store the performance for this generation
    success_rate = successful_interactions / num_interactions_dynamic
    performance_over_generations_dynamic.append(success_rate)
    
    # Apply the growth rule to increase the population
    grow_population(senders_population, receivers_population)
    
    # Track the population size
    population_size = len(senders_population)  # or len(receivers_population), they are the same
    population_size_over_generations.append(population_size)
    print(f"Generation {generation + 1}, Success Rate: {success_rate:.4f}, Population Size: {population_size}")


/var/folders/j9/r2v28fnx719_1zz4f5yk8gbh0000gn/T/ipykernel_33012/2139423504.py:51: UserWarning: Using a target size (torch.Size([1, 28, 28])) that is different to the input size (torch.Size([1, 1, 28, 28])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(decoded_image, original_image)


Generation 1, Success Rate: 0.0000, Population Size: 4
Generation 2, Success Rate: 0.0000, Population Size: 5
Generation 3, Success Rate: 0.0000, Population Size: 6
Generation 4, Success Rate: 0.0000, Population Size: 7
Generation 5, Success Rate: 0.0000, Population Size: 8
Generation 6, Success Rate: 0.0000, Population Size: 9
Generation 7, Success Rate: 0.0000, Population Size: 10
Generation 8, Success Rate: 0.0000, Population Size: 11
Generation 9, Success Rate: 0.0000, Population Size: 12
Generation 10, Success Rate: 0.0000, Population Size: 13
Generation 11, Success Rate: 0.0000, Population Size: 14
Generation 12, Success Rate: 0.0000, Population Size: 15
Generation 13, Success Rate: 0.0000, Population Size: 16
Generation 14, Success Rate: 0.0000, Population Size: 17
Generation 15, Success Rate: 0.0000, Population Size: 18
Generation 16, Success Rate: 0.0000, Population Size: 19
Generation 17, Success Rate: 0.0000, Population Size: 20
Generation 18, Success Rate: 0.0000, Populatio